In [1]:
import psutil
import matplotlib.pyplot as plt
from gtts import gTTS
import pygame
import time
import dropbox
from dropbox.exceptions import AuthError
from dropbox.files import WriteMode

# Replace with your Dropbox access token
ACCESS_TOKEN = "access token"

# Get CPU usage
cpu_usage = psutil.cpu_percent(interval=1)

# Generate bar chart
plt.bar(['CPU Usage'], [cpu_usage], color='blue')
plt.ylim(0, 100)
plt.ylabel('Percentage (%)')
plt.title('System CPU Usage')
local_png = '/home/aivie/cpu_usage.png'
plt.savefig(local_png)
plt.close()

# Generate MP3
text = f"The current CPU usage is {cpu_usage} percent."
local_mp3 = '/home/aivie/cpu_usage.mp3'
tts = gTTS(text=text, lang='en')
tts.save(local_mp3)

# Try to play audio in VM (optional, may fail due to sound card)
pygame.mixer.pre_init(44100, -16, 2, 2048)
pygame.mixer.init()
try:
    pygame.mixer.music.load(local_mp3)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        time.sleep(1)
except pygame.error as e:
    print(f"Pygame error: {e}")

# Upload to Dropbox
dbx = dropbox.Dropbox(ACCESS_TOKEN)
try:
    with open(local_png, 'rb') as f:
        dbx.files_upload(f.read(), '/cpu_usage.png', mode=WriteMode('overwrite'), mute=True)
    with open(local_mp3, 'rb') as f:
        dbx.files_upload(f.read(), '/cpu_usage.mp3', mode=WriteMode('overwrite'), mute=True)
    print("Files uploaded to Dropbox successfully (overwritten if existed)!")
except AuthError as e:
    print(f"Dropbox auth error: {e}")
except Exception as e:
    print(f"Upload error: {e}")

print(f"CPU Usage: {cpu_usage}%")

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Files uploaded to Dropbox successfully (overwritten if existed)!
CPU Usage: 6.8%
